逻辑回归常用于医学上研究多个影响因素对结果变量的关系或作用。例如：冠心病与高血压史、高血脂史和吸烟等危险因素关系的研究

参考网页 ： http://www.woc88.com/p32705383.html

           厦门大学数据库实验室 ： http://dblab.xmu.edu.cn/blog/logistic-regression-in-python/

采用单因素统计方法：
    - 常采用X2卡方检验，但如有混杂因素干扰，会导致结果不一定正确；
    - 不能回答哪个因素（x）对（y）关系更密切或作用更大；
采用多因素分析：
    - 常常使用逻辑回归，可校正混杂因素，正确评价结果的效应；
    - 回答哪个因素对事件（疾病）作用更大；
    
<b> 约束条件 ： 自变量之间是非共线性的。 </b>
    
<center><img src='data/logistics.png' /> </center>



In [40]:
#加载需要的包
from sklearn.linear_model import LogisticRegression
import os
import pandas as pd
import seaborn as sns
import numpy as np
import scipy
from scipy import stats
import statsmodels.api as sm

# <b> 准备数据 </b>

In [41]:
dir = "./"
print(os.listdir("./"))  # 确认路径正确

['.ipynb_checkpoints', 'data', 'install.ipynb', '10. ROC分析.ipynb', '02 . 统计描述.ipynb', '01. 绪论.ipynb', '04. 两组数值变量比较的假设检验.ipynb', '05. 多组数值变量比较的假设检验.ipynb', '06. 有序变量比较的假设检验.ipynb', '07. 分类变量比较的假设检验.ipynb', '08. 相关性假设检验.ipynb', '03. 常规检验.ipynb', '09. 多元回归析.ipynb']


In [42]:
df = pd.read_csv(dir+"data/心脏病诊断数据集（原数据）.csv",header=None) #读取csv文件，没有列名
df.columns = ["年龄","性别","胸痛类型","静息血压","血浆类固醇含量（mg/dl）","空腹血糖>120mg/dl","静息心电图结果","最高心率","运动型心绞痛","运动引起的ST下降","最大运动量时心电图ST的斜率","使用荧光染色法测定的主血管数","THAL","患病情况"]                                                 #添加列名
df.head(5)

,年龄,性别,胸痛类型,静息血压,血浆类固醇含量（mg/dl）,空腹血糖>120mg/dl,静息心电图结果,最高心率,运动型心绞痛,运动引起的ST下降,最大运动量时心电图ST的斜率,使用荧光染色法测定的主血管数,THAL,患病情况
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,2
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,1
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,2
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,1
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,1


In [43]:
print (df.describe())
print (df.std())

               年龄          性别        胸痛类型        静息血压  血浆类固醇含量（mg/dl）  \
count  270.000000  270.000000  270.000000  270.000000      270.000000   
mean    54.433333    0.677778    3.174074  131.344444      249.659259   
std      9.109067    0.468195    0.950090   17.861608       51.686237   
min     29.000000    0.000000    1.000000   94.000000      126.000000   
25%     48.000000    0.000000    3.000000  120.000000      213.000000   
50%     55.000000    1.000000    3.000000  130.000000      245.000000   
75%     61.000000    1.000000    4.000000  140.000000      280.000000   
max     77.000000    1.000000    4.000000  200.000000      564.000000   

       空腹血糖>120mg/dl     静息心电图结果        最高心率      运动型心绞痛  运动引起的ST下降  \
count     270.000000  270.000000  270.000000  270.000000  270.00000   
mean        0.148148    1.022222  149.677778    0.329630    1.05000   
std         0.355906    0.997891   23.165717    0.470952    1.14521   
min         0.000000    0.000000   71.000000    0.000000  

In [44]:
df.患病情况 = df.患病情况-1   #将目标变量转变成0,1.否则会报错。 ValueError: endog must be in the unit interval.
print (np.unique(df.患病情况))

[0 1]


<b> 虚拟变量（dummy variables）</b>

In [45]:
classfield =  ['性别',"胸痛类型","空腹血糖>120mg/dl","静息心电图结果","运动型心绞痛","最大运动量时心电图ST的斜率"] #现将分类变量转成字符型
df[classfield] = df[classfield].astype("str")   #将分类变量转成字符型
dummys = df.filter(items = classfield)  
dummy_ranks = pd.get_dummies(dummys , drop_first = True) # 变成哑变量，并且丢掉第一个分类
cols_to_keep= ["患病情况","年龄","静息血压","血浆类固醇含量（mg/dl）" ,"最高心率","运动引起的ST下降","使用荧光染色法测定的主血管数","THAL"]
data = df[cols_to_keep].join(dummy_ranks)     #将新的虚拟变量加入到原数据集中
print (data.head())

   患病情况  年龄  静息血压  血浆类固醇含量（mg/dl）  最高心率  运动引起的ST下降  使用荧光染色法测定的主血管数  THAL  \
0     1  70   130             322   109        2.4               3     3   
1     0  67   115             564   160        1.6               0     7   
2     1  57   124             261   141        0.3               0     7   
3     0  64   128             263   105        0.2               1     7   
4     0  74   120             269   121        0.2               1     3   

   性别_1  胸痛类型_2  胸痛类型_3  胸痛类型_4  空腹血糖>120mg/dl_1  静息心电图结果_1  静息心电图结果_2  \
0     1       0       0       1                0          0          1   
1     0       0       1       0                0          0          1   
2     1       1       0       0                0          0          0   
3     1       0       0       1                0          0          0   
4     0       1       0       0                0          0          1   

   运动型心绞痛_1  最大运动量时心电图ST的斜率_2  最大运动量时心电图ST的斜率_3  
0         0                 1                 0 

# <b>   训练模型  </b>

In [46]:
train_cols = data.columns[1:]                        #自变量的列名
print(train_cols)
logit = sm.Logit(data['患病情况'], data[train_cols])  #模型
result = logit.fit()                               #拟合

Index(['年龄', '静息血压', '血浆类固醇含量（mg/dl）', '最高心率', '运动引起的ST下降', '使用荧光染色法测定的主血管数',
       'THAL', '性别_1', '胸痛类型_2', '胸痛类型_3', '胸痛类型_4', '空腹血糖>120mg/dl_1',
       '静息心电图结果_1', '静息心电图结果_2', '运动型心绞痛_1', '最大运动量时心电图ST的斜率_2',
       '最大运动量时心电图ST的斜率_3'],
      dtype='object')
Optimization terminated successfully.
         Current function value: 0.330390
         Iterations 7


# <b> 结果解释 </b>

##  <b> 系数显著性水平 </b>

H0 ： c == 0  H1 : c != 0  

(c为变量系数，当推断c为零时，则在统计意义上变是不是影响因素，c不为零，则是影响因素）

Z检验（Z Test）是一般用于大样本（即样本容量大于30）平均值差异性检验的方法。它是用标准正态分布的理论来推断差异发生的概率，从而比较两个平均数的差异是否显著。在国内也被称作u检验。
当已知标准差时，验证一组数的均值是否与某一期望值相等时，用Z检验。


In [51]:
print (result.summary())
# 查看每个系数的置信区间：
# print (result.conf_int())  

                           Logit Regression Results                           
Dep. Variable:                   患病情况   No. Observations:                  270
Model:                          Logit   Df Residuals:                      253
Method:                           MLE   Df Model:                           16
Date:                Fri, 21 Jun 2019   Pseudo R-squ.:                  0.5191
Time:                        09:33:21   Log-Likelihood:                -89.205
converged:                       True   LL-Null:                       -185.48
                                        LLR p-value:                 2.530e-32
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
年龄                  -0.0534      0.022     -2.390      0.017      -0.097      -0.010
静息血压                 0.0139      0.011      1.295      0.195      -0.007       0.035
血浆类固醇含量（mg/dl）       0.0054 

从上面来看THAL（p = 0.003 ）是患病的危险因子。

## <b>  相对危险度（odds ratio）</b>

使用每个变量系数的指数来生成odds ratio，可知变量每单位的增加、减少对患病几率的影响。

In [54]:
print(np.exp(result.params))

年龄                  0.948043
静息血压                1.013985
血浆类固醇含量（mg/dl）      1.005402
最高心率                0.963884
运动引起的ST下降           1.513411
使用荧光染色法测定的主血管数      3.186369
THAL                1.392165
性别_1                3.388964
胸痛类型_2              1.808351
胸痛类型_3              0.918175
胸痛类型_4              4.722888
空腹血糖>120mg/dl_1     0.765932
静息心电图结果_1           1.925363
静息心电图结果_2           1.785838
运动型心绞痛_1            1.713001
最大运动量时心电图ST的斜率_2    1.757476
最大运动量时心电图ST的斜率_3    0.957739
dtype: float64


从结果来看性别_1风险比较高，即男性比女性更容易患病。

我们也可以使用置信区间来计算系数的影响，来更好地估计一个变量影响患病几率的不确定性。

In [57]:
# odds ratios and 95% CI
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print (np.exp(conf))

                      2.5%       97.5%        OR
年龄                0.907461    0.990441  0.948043
静息血压              0.992895    1.035522  1.013985
血浆类固醇含量（mg/dl）    0.997203    1.013669  1.005402
最高心率              0.947451    0.980602  0.963884
运动引起的ST下降         0.949660    2.411823  1.513411
使用荧光染色法测定的主血管数    1.882174    5.394266  3.186369
THAL              1.121013    1.728904  1.392165
性别_1              1.237709    9.279302  3.388964
胸痛类型_2            0.377088    8.672073  1.808351
胸痛类型_3            0.246459    3.420637  0.918175
胸痛类型_4            1.317213   16.933996  4.722888
空腹血糖>120mg/dl_1   0.242252    2.421659  0.765932
静息心电图结果_1         0.004936  751.037306  1.925363
静息心电图结果_2         0.810767    3.933580  1.785838
运动型心绞痛_1          0.715852    4.099129  1.713001
最大运动量时心电图ST的斜率_2  0.706296    4.373132  1.757476
最大运动量时心电图ST的斜率_3  0.141051    6.503060  0.957739


# <b> 预测数据 </b>

可见参考页面：http://dblab.xmu.edu.cn/blog/logistic-regression-in-python/
